In [ ]:
# TESTING WITH LATEST DATASET (00:00, 2, December, 2025) -> (00:00, 06, December, 2025)

In [17]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone

PROM_URL = "http://10.35.29.108:30900/api/v1/query_range"

# TIME RANGE (THAILAND)
TH = timezone(timedelta(hours=7))

start = datetime(2025, 12, 2, 0, 0, 0, tzinfo=TH)
end   = datetime(2025, 12, 6, 0, 0, 0, tzinfo=TH)

start_ts = start.timestamp()
end_ts = end.timestamp()

# NODES
nodes = {
    "109": "10.35.29.109:9100",
    "110": "10.35.29.110:9100"
}

# RAW PROMQL QUERIES
CPU_QUERY = 'node_cpu_seconds_total{instance="%s"}'
MEM_AVAIL_QUERY = 'node_memory_MemAvailable_bytes{instance="%s"}'
MEM_TOTAL_QUERY = 'node_memory_MemTotal_bytes{instance="%s"}'

RUNNING_QUERY = 'sum(kube_pod_status_phase{phase="Running"})'
PENDING_QUERY = 'sum(kube_pod_status_phase{phase="Pending"})'

# GENERIC QUERY FUNCTION
def prom_query(query):
    params = {
        "query": query,
        "start": start_ts,
        "end": end_ts,
        "step": "60s"
    }

    r = requests.get(PROM_URL, params=params).json()

    if "data" not in r or len(r["data"]["result"]) == 0:
        return pd.DataFrame()

    frames = []
    for result in r["data"]["result"]:

        df = pd.DataFrame(result["values"], columns=["timestamp", "value"])
        
        # Convert to Thailand time
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s", utc=True)
        df["timestamp"] = df["timestamp"].dt.tz_convert("Asia/Bangkok")

        df["value"] = df["value"].astype(float)

        # Attach metric labels
        for k, v in result.get("metric", {}).items():
            df[k] = v

        frames.append(df)

    return pd.concat(frames, ignore_index=True)

# QUERY RAW CPU + MEMORY
cpu_frames = []
mem_avail_frames = []
mem_total_frames = []

for node_id, inst in nodes.items():

    # CPU counters
    raw_cpu = prom_query(CPU_QUERY % inst)
    raw_cpu["node"] = node_id
    cpu_frames.append(raw_cpu)

    # Memory available
    raw_avail = prom_query(MEM_AVAIL_QUERY % inst)
    raw_avail = raw_avail.rename(columns={"value": "mem_avail"})
    raw_avail["node"] = node_id
    mem_avail_frames.append(raw_avail)

    # Memory total
    raw_total = prom_query(MEM_TOTAL_QUERY % inst)
    raw_total = raw_total.rename(columns={"value": "mem_total"})
    raw_total["node"] = node_id
    mem_total_frames.append(raw_total)


cpu_raw = pd.concat(cpu_frames)
mem_avail = pd.concat(mem_avail_frames)
mem_total = pd.concat(mem_total_frames)

# Running (cluster-wide)
running_raw = prom_query(RUNNING_QUERY).rename(columns={"value": "running_total"})

# Pending (cluster-wide)
pending_raw = prom_query(PENDING_QUERY).rename(columns={"value": "pending"})

# PIVOT CPU COUNTERS
cpu_pivot = cpu_raw.pivot_table(
    index="timestamp",
    columns=["node", "mode"],
    values="value"
)

cpu_pivot.columns = [
    f"cpu_{mode}_{node}"
    for (node, mode) in cpu_pivot.columns
]

cpu_pivot = cpu_pivot.reset_index()

# MERGE MEMORY + POD METRICS
df = cpu_pivot

# memory available
df = df.merge(
    mem_avail.pivot_table(index="timestamp", columns="node", values="mem_avail")
        .rename(columns=lambda x: f"mem_avail_{x}"),
    on="timestamp", how="left"
)

# memory total
df = df.merge(
    mem_total.pivot_table(index="timestamp", columns="node", values="mem_total")
        .rename(columns=lambda x: f"mem_total_{x}"),
    on="timestamp", how="left"
)

# running cluster-wide
df = df.merge(running_raw, on="timestamp", how="left")

# pending cluster-wide
df = df.merge(pending_raw, on="timestamp", how="left")

df = df.sort_values("timestamp")

df.to_csv("Dataset_Dec2_Dec6.csv", index=False)

print("✔ DONE — Final dataset created:", df.shape)
print(df.head())


✔ DONE — Final dataset created: (5761, 23)
                  timestamp  cpu_idle_109  cpu_iowait_109  cpu_irq_109  \
0 2025-12-02 00:00:00+07:00  1.666666e+06        357.6225          0.0   
1 2025-12-02 00:01:00+07:00  1.666676e+06        357.6425          0.0   
2 2025-12-02 00:02:00+07:00  1.666689e+06        357.6450          0.0   
3 2025-12-02 00:03:00+07:00  1.666701e+06        357.6500          0.0   
4 2025-12-02 00:04:00+07:00  1.666713e+06        357.6500          0.0   

   cpu_nice_109  cpu_softirq_109  cpu_steal_109  cpu_system_109  cpu_user_109  \
0      130.5175        1017.5200            0.0      84293.2350   861675.2775   
1      130.5425        1017.5700            0.0      84297.4650   861718.4125   
2      130.5425        1017.6225            0.0      84301.4700   861759.6100   
3      130.5425        1017.6700            0.0      84305.3350   861801.4075   
4      130.5425        1017.7100            0.0      84309.4125   861843.1800   

   cpu_idle_110  ...  cpu